In [1]:
import sys

sys.path.append("/home/jupyter-aamir09/mlops2_with_dagster/")


from mlops2_with_dagster import encoder_pipeline
from pathlib import Path
from joblib import dump, load
from hamilton import driver, base
from mlops2_with_dagster.utils import get_project_dir, printse

from sklearn.preprocessing import (
    StandardScaler,
    LabelEncoder
)
from ortho.utils.logger import Logger
from ortho.ortho.decorators.task import task
from typing import Tuple

import pandas as pd 
import mlflow as mf
from ortho.ortho.callbacks.mlflow import MlFlowCallBack
from ortho.ortho.callbacks.logger import LoggerCallBack
import os


logger = Logger().logger()


# class MyProcessor(PreProcessor):
    
#     def __init__(self, callbacks,  experiment_name, run_name, load_from_artifact=False):
#         self.load_from_artifact = load_from_artifact
#         self.callbacks = callbacks
#         self.experiment_name = experiment_name
#         self.run_name = run_name
#         os.environ["MLFLOW_EXPERIMENT_NAME"] = self.experiment_name
#         os.environ["MLFLOW_RUN_NAME"] = self.run_name
    
#     @task(end_mlflow_run=True)
#     def train_processor(self, training_data):
        
    #     data, _ , params = preprocess(training_data.copy(), training_data.copy())
        
    #     payload = {"artifact_path":"/home/jupyter-aamir09/dagster_pipeline_2/artefacts"}
        
    #     return {"train_x":data.drop("target", axis =1),
    #             "train_y": data["target"], 
    #             "processor_params": params}, payload
    
    # def inference_preprocessor(self, infer_data, params):
    
    #     data = inference_preprocess(infer_data.copy(), params)
        

        # payload = {"artifacts_path":"/home/jupyter-aamir09/dagster_pipeline_2/artefacts"}
        
        # return {"inference_data":data}, payload
        
        
        
# if __name__ == "__main__":
    
#     data_path = "/home/jupyter-aamir09/dagster_pipeline_2/data/raw/train.csv"
    
#     data = pd.read_csv(data_path)
    
#     processor = MyProcessor(filenames=[], 
#                             callbacks=[MlFlowCallBack(kernel_names=["train_processor"])])
    
#     processor.train_processor(data)

None


2023-08-30 19:07:25.746144: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-30 19:07:25.780683: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-30 19:07:25.781272: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-30 19:07:26,271 - DEBUG - Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
2023-08-30 19:07:26.324683: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-30 19:07:26,451 - DEBUG - Creating converter from 7 to 5
2023-08-30 19:07:26,452 - DEBUG - Creating converter from 

In [2]:
project = 'mlops2_with_dagster'
project_dir = get_project_dir(project)
printse(f'project_dir: {project_dir}')

>>>> project_dir: /home/jupyter-aamir09/mlops2_with_dagster


In [3]:
class Encoder:
    
    def __init__(self, callbacks,  experiment_name, run_name, load_from_artifact=False):
        self.load_from_artifact = load_from_artifact
        self.callbacks = callbacks
        self.experiment_name = experiment_name
        self.run_name = run_name
        os.environ["MLFLOW_EXPERIMENT_NAME"] = self.experiment_name
        os.environ["MLFLOW_RUN_NAME"] = self.run_name
        
        self.index_col = 'passengerid'
        self.target_col = "survived"
        self.cat_cols = ["sex", "cabin", "embarked"]
        self.config = {
            'index_column': self.index_col,
            'target_column': self.target_col,
            'categorical_columns': self.cat_cols
        }
    
    @task()       
    def create_encoders(self, df_train, df_test):
        adapter = base.SimplePythonGraphAdapter(base.DictResult())
        encode_dr = driver.Driver(self.config, encoder_pipeline, adapter=adapter)
        output_nodes = ['encoders']
        
        out = encode_dr.execute(output_nodes ,
        inputs = dict(
            df_train = df_train,
            df_test = df_test
        )         
        )
        
        data = {"encoders": out }
        payload = {"params":self.config,
                   "artifact_path": "/home/jupyter-aamir09/mlops2_with_dagster/artifacts/mlflow_artfacts"}
        
        return data, payload
    
    def run(self, *args, **kwargs):
        
        return self.create_encoders(*args, **kwargs)
        
        
    

In [4]:
train_data : Path = project_dir/"data/train.csv"
test_data : Path = project_dir/"data/test.csv"

In [5]:
# parameters
df_train = pd.read_csv(train_data)
df_test = pd.read_csv(test_data)

In [6]:
import dagstermill
encoders_object = Encoder(experiment_name="Mlflow_with_Dagster",
                          run_name="BadamBhum",
                          callbacks = [LoggerCallBack("/home/jupyter-aamir09/mlops2_with_dagster/notebooks", kernel_names=["create_encoders"]),
                                       MlFlowCallBack(kernel_names=["create_encoders"])])

mf.end_run()
dagstermill.yield_result(encoders_object.run(df_train=df_train,
                                            df_test=df_test)[0]["encoders"], output_name="encoders")

2023-08-30 19:07:27,113 - DEBUG - Starting new HTTP connection (1): 127.0.0.1:5002
2023-08-30 19:07:27,122 - DEBUG - http://127.0.0.1:5002 "GET /api/2.0/mlflow/experiments/get-by-name?experiment_name=Mlflow_with_Dagster HTTP/1.1" 200 241
2023-08-30 19:07:27,146 - DEBUG - Popen(['git', 'version'], cwd=/home/jupyter-aamir09/mlops2_with_dagster/notebooks, universal_newlines=False, shell=None, istream=None)
2023-08-30 19:07:27,149 - DEBUG - Popen(['git', 'version'], cwd=/home/jupyter-aamir09/mlops2_with_dagster/notebooks, universal_newlines=False, shell=None, istream=None)
2023-08-30 19:07:27,154 - DEBUG - Failed checking if running in CYGWIN due to: FileNotFoundError(2, 'No such file or directory')
2023-08-30 19:07:27,157 - DEBUG - Resetting dropped connection: 127.0.0.1
2023-08-30 19:07:27,169 - DEBUG - http://127.0.0.1:5002 "POST /api/2.0/mlflow/runs/create HTTP/1.1" 200 918
2023-08-30 19:07:27,170 - INFO - Successfuly initiated run with name BadamBhum and id 1979dcd81bfb4588a25f196d619

['self', 'df_train', 'df_test']


2023-08-30 19:07:31,199 - DEBUG - Retry: /api/2.0/mlflow-artifacts/artifacts/1/1979dcd81bfb4588a25f196d619475b5/artifacts/df_train.pickle
2023-08-30 19:07:31,201 - DEBUG - Resetting dropped connection: 127.0.0.1
2023-08-30 19:07:31,206 - DEBUG - http://127.0.0.1:5002 "GET /api/2.0/mlflow-artifacts/artifacts/1/1979dcd81bfb4588a25f196d619475b5/artifacts/df_train.pickle HTTP/1.1" 500 258
2023-08-30 19:07:31,207 - DEBUG - Incremented Retry for (url='/api/2.0/mlflow-artifacts/artifacts/1/1979dcd81bfb4588a25f196d619475b5/artifacts/df_train.pickle'): Retry(total=2, connect=5, read=5, redirect=5, status=2)
2023-08-30 19:07:39,216 - DEBUG - Retry: /api/2.0/mlflow-artifacts/artifacts/1/1979dcd81bfb4588a25f196d619475b5/artifacts/df_train.pickle
2023-08-30 19:07:39,218 - DEBUG - Resetting dropped connection: 127.0.0.1
2023-08-30 19:07:39,222 - DEBUG - http://127.0.0.1:5002 "GET /api/2.0/mlflow-artifacts/artifacts/1/1979dcd81bfb4588a25f196d619475b5/artifacts/df_train.pickle HTTP/1.1" 500 258
2023-0

{}


2023-08-30 19:09:27,656 - DEBUG - Succeed in sending telemetry consisting of [b'{"api_key": "phc_mZg8bkn3yvMxqvZKRlMlxjekFU5DFDdcdAsijJ2EH5e", "event": "os_hamilton_run_start", "properties": {"os_type": "posix", "os_version": "Linux-5.4.0-124-generic-x86_64-with-glibc2.31", "python_version": "3.10.12/CPython", "distinct_id": "e4e272c1-5845-4b12-8bd8-09d84d585dbc", "hamilton_version": [1, 27, 2], "telemetry_version": "0.0.1", "number_of_nodes": 32, "number_of_modules": 1, "number_of_config_items": 3, "decorators_used": {"parameterize_sources": 4, "extract_columns": 2, "extract_fields": 1}, "graph_adapter_used": "hamilton.base.SimplePythonGraphAdapter", "result_builder_used": "hamilton.base.DictResult", "driver_run_id": "f7c6265c-4894-4121-b201-85574ec39aa8", "error": null, "graph_executor_class": "DefaultGraphExecutor"}}'].
2023-08-30 19:09:27,696 - DEBUG - Succeed in sending telemetry consisting of [b'{"api_key": "phc_mZg8bkn3yvMxqvZKRlMlxjekFU5DFDdcdAsijJ2EH5e", "event": "os_hamilton_

{'encoders': {'cabinencoder': LabelEncoder(),
  'sexencoder': LabelEncoder(),
  'embarkedencoder': LabelEncoder()}}